<a href="https://colab.research.google.com/github/varshaelza/Text-Summarization-Categorization/blob/main/LDASumm2_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def sep(text):
  import pandas as pd
  import nltk
  nltk.download('punkt')
  import re
  text= text.replace("\t"," ")
  text=re.sub(r'\.+', '.',text)
  k=0
  c=0
  p=0
  a_list=[]
  while k<len(text):
    if c==0 and text[k]=='.':
      a_list.append(text[p:k+1])
      p=k+1
    elif c==0 and text[k]=='"':
      c=1
    elif c==1 and text[k]=='"' and text[k-1]=="." :
      a_list.append(text[p:k+1])
      p=k+1
      c=0
    elif c==1 and text[k]=='"' and text[k-2]=="." :
      a_list.append(text[p:k+1])
      p=k+1
      c=0
    elif c==1 and text[k]=='"':
      c=0
    k=k+1
  if p<len(text)-1:
    a_list.append(text[p:len(text)])
 # a_list = nltk.tokenize.sent_tokenize(text)
  #s=len(a_list)
  df=pd.DataFrame()
  print(a_list[0])
  j=0
  df['News']=""
  while j<len(a_list):
    if j!=0:
      df.loc[len(df.index)]=[a_list[j].replace("\n","")]
    else:
      df.loc[len(df.index)]=[a_list[j]]
    j=j+1
  return df

def spl(text):
  t=text.split('\n',1)
  if len(t)!=2:
    return ""
  else :
    return t[1]

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#print(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
  import nltk
  nltk.download('stopwords')
  from nltk.corpus import stopwords
  STOPWORDS = set(stopwords.words('english'))
  """custom function to remove the stopwords"""
  return " ".join([word for word in str(text).split() if word not in STOPWORDS])


def lemmatize_words(text):
  
  from nltk.stem import WordNetLemmatizer
  import nltk
  nltk.download('wordnet')
  lemmatizer = WordNetLemmatizer()
  return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


def remove_punctuation(text):
    """custom function to remove the punctuation"""
    import string
    PUNCT_TO_REMOVE =  '''!()-[]{};:'"\,<>/?@#$%^&*_~'''
    no_punct = ""
    for char in text:
      if char not in PUNCT_TO_REMOVE:
        no_punct = no_punct + char
      else:
        no_punct = no_punct + ' '
    return(no_punct)

   
def remove_stop(text):
    """custom function to remove the stopwords"""
    STOPWORDS = ['a' ,'b', 'c','d','e','f' ,'g' ,'h','i','j','k','l','m','n','o' ,'p' ,'q','r','s','t','u' ,'v' ,'w','x','y','z']
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])



def pre(df):
  df["News(wh)"]=df['News']
  df["News(wh)"][0]=spl(df["News"][0])
  df["News(wh)"][0]=spl(df["News(wh)"][0])
  df["News1"] = df["News(wh)"].str.lower()
  df['News1'] = df['News1'].replace('\n','.', regex=True)
  df['News1'] = df['News1'].replace('\t',' ', regex=True)
  df['News1'] = df['News1'].replace(r'\.+', ".", regex=True)
  df["News1"] = df["News1"].apply(lambda text: remove_stopwords(text))
  df['News1'] = df['News1'].str.replace('(\d*\.\d+)|(\d+\.[0-9 ]+)|(\d)', '')
  df['News1'] = df['News1'].apply(lambda text: lemmatize_words(text))
  df['News1'] = df['News1'].apply(lambda text: remove_punctuation(text))
  df['News1'] = df['News1'].replace(r'\.+', ".", regex=True)
  df['News1'] = df['News1'].apply(lambda text: remove_stop(text))
  return df
  

def lemmatization(texts, tags=['NOUN', 'ADJ','VERB','ADV']): # filter noun and adjective
       import spacy
       nlp = spacy.load('en', disable=['parser', 'ner'])
       output = []
       for sent in texts:
             doc = nlp(" ".join(sent)) 
             output.append([token.lemma_ for token in doc if token.pos_ in tags])
       return output

def preprocess(text):
  import gensim
  result = []
  for token in gensim.utils.simple_preprocess(text):
    result.append(token)
  return result



#for five category topic model 

def format_topics_sentencesf(ldamodel, corpus, texts,docs):
    # Init output
    import pandas as pd
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if prop_topic >=.1 :  # => topics to include
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords,docs['News(wh)'][i]]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Topic', 'Perc_Contribution', 'Topic_Keywords','Sentence']

    return sent_topics_df



def scoringf(df,k):
  import gensim
  import joblib
  from nltk.stem import WordNetLemmatizer, SnowballStemmer
  from gensim.utils import simple_preprocess
  from gensim.parsing.preprocessing import STOPWORDS
  processed_docs = df['News1'].map(preprocess)
  processed_docs = processed_docs.values.tolist()
  processed_docs=lemmatization(processed_docs)
  if k==0:
    dictionary=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/buss_dict.jl')
    lda_model=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/buss_opttfidf_genlda.jl')
  elif k==1:
    dictionary=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/ent_dict.jl')
    lda_model=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/ent_opttfidf_genlda.jl')
  elif k==2:
    dictionary=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/pol_dict.jl')
    lda_model=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/pol_opttfidf_genlda.jl')
  elif k==3:
    dictionary=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/spo_dict.jl')
    lda_model=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/spo_opttfidf_genlda.jl')
  elif k==4:
    dictionary=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/tech_dict.jl')
    lda_model=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/tech_opttfidf_genlda.jl')
  bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
  from gensim import corpora, models
  tfidf = models.TfidfModel(bow_corpus)
  corpus_tfidf = tfidf[bow_corpus]
  df1 = format_topics_sentencesf(ldamodel=lda_model, corpus=corpus_tfidf, texts=processed_docs,docs=df)
  return df1

#for all category topic model 

def format_topics_sentences(ldamodel, corpus, texts,docs):
    # Init output
    import pandas as pd
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if prop_topic >=.1 :  # => topics to include
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords,docs['News(wh)'][i]]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Topic', 'Perc_Contribution', 'Topic_Keywords','Sentence']

    return sent_topics_df

def scoring(df):
  import gensim
  import joblib
  from nltk.stem import WordNetLemmatizer, SnowballStemmer
  from gensim.utils import simple_preprocess
  from gensim.parsing.preprocessing import STOPWORDS
  processed_docs = df['News1'].map(preprocess)
  processed_docs = processed_docs.values.tolist()
  processed_docs=lemmatization(processed_docs)
  dictionary=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/all_dict.jl')
  lda_model=joblib.load('/content/drive/MyDrive/BBC Dataset/LDA Summarization/all_opttfidf_genlda.jl')
 
  bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
  from gensim import corpora, models
  tfidf = models.TfidfModel(bow_corpus)
  corpus_tfidf = tfidf[bow_corpus]
  df1 = format_topics_sentences(ldamodel=lda_model, corpus=corpus_tfidf, texts=processed_docs,docs=df)
  return df1



#import pandas as pd
#train = pd.read_csv('/content/drive/MyDrive/BBC Dataset/LDA Summarization/LDASummariestrain.csv')
#train['Summary1(Imp)']=""
#train['Length1(Imp)']=0
#i=0

#while i< len(train):
def getsummary(text,w):
  import pandas as pd
  df=sep(text)
  df=pre(df)
  df1=scoringf(df,w)
  j=1;
  s=df['News'][0].split('\n',1)[0]
  s=s+'\n'
  c=1
  l=0
  df1['Count']=0
  df1['Diff']=-10000.000
  while j<len(df1):
    if df1['Sentence'][j]==df1['Sentence'][j-1]:
      #print(j)
      c=c+1
    else:
      #if c>3:
       # print(df1['Sentence'][j-1])
        #s=s+df1['Sentence'][j-1]
        df1['Count'][j-1]=c
        df1['Diff'][j-1]=float(df1['Perc_Contribution'][j-1]-df1['Perc_Contribution'][j-c])
        c=1
        #l=l+1
    j=j+1

  df1['Count'][j-1]=c 
  df1['Diff'][j-1]=float(df1['Perc_Contribution'][j-1]-df1['Perc_Contribution'][j-c])
  
  dup=df1.copy(deep=True)      

  dup.sort_values(by=['Count','Diff'],inplace=True,ascending=False)
  k=dup.head(5)
  
  p=0
  lg=0
  while l<5 and l<len(k):
    try:
      if k['Count'][p]!=1 and k['Count'][p]!=0:
        s=s+k['Sentence'][p]
        lg=lg+1
      p=p+1
      l=l+1
    except:
      p=p+1


  
  #train['Summary1(Imp)'][i]=s
  #train['Length1(Imp)'][i]=lg
  #i=i+1
  return s



